In [2]:
import importlib
import src.chat_bot
importlib.reload(src.chat_bot)
from src.chat_bot import ExconManual

In [5]:
#ad = True
ad = False

logging_level = 15
log_file = ''

if ad:
    path_to_manual_as_csv_file = "./inputs/ad_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/ad_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/ad_index.parquet"
    excon = ExconManual(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, log_file=log_file, logging_level=logging_level)
else:
    path_to_manual_as_csv_file = "./inputs/adla_manual.csv"
    path_to_definitions_as_parquet_file = "./inputs/adla_definitions.parquet"
    path_to_index_as_parquet_file = "./inputs/adla_index.parquet"
    excon = ExconManual(path_to_manual_as_csv_file, path_to_definitions_as_parquet_file, path_to_index_as_parquet_file, chat_for_ad = False,log_file=log_file, logging_level=logging_level)


In [12]:
import pandas as pd
#df_definitions_all = pd.read_parquet(path_to_definitions_as_parquet_file, engine='pyarrow')
df_text_all = pd.read_parquet(path_to_index_as_parquet_file, engine='pyarrow')

In [13]:
#df_definitions_all.columns
df_text_all

,section,text,source,Embedding
0,A.3(A),What are the different categories of an ADLA a...,question,"[0.01191499549895525, 0.005717828404158354, -3..."
1,A.3(A),What are some services an ADLA can provide?,question,"[0.017316650599241257, 0.012317745015025139, -..."
2,A.3(A),Can all ADLAs operate a Bureau de Change or pr...,question,"[-0.00110212666913867, -0.004164155572652817, ..."
3,A.3(B)(i),What is the process to become an Authorised De...,question,"[0.022449666634202003, -0.009842596016824245, ..."
4,A.3(B)(i),Who approves ADLAs?,question,"[0.016875632107257843, -0.007422278635203838, ..."
...,...,...,...,...
320,C.1(H)(i),FinSurv Reporting System. Balance-of-payments ...,heading,"[-0.01508619636297226, 0.00018646813987288624,..."
321,C.1(H)(ii),FinSurv Reporting System. Balance-of-payments ...,heading,"[-0.00906417891383171, -0.011498916894197464, ..."
322,C.1(H)(iii),FinSurv Reporting System. Balance-of-payments ...,heading,"[-0.01566045545041561, -0.005427613854408264, ..."
323,C.1(I),FinSurv Reporting System. Reconciliation module,heading,"[-0.006933209020644426, 0.0037072172854095697,..."


In [3]:
model_to_use="gpt-3.5-turbo"
#model_to_use="gpt-4"
#model_to_use="gpt-3.5-turbo-16k"

In [5]:
user_context = "A non-resident wants to buy residential property in South Africa. What are the restrictions around doing this?"
#user_context = "A South African citizen is emigrating. What is the process they need to follow to take all their cash offshore?"
#user_context = "What is the Foreign investment limits for South African institutional investors?"
#user_context = "Can a individual transfer his shares in a dual listed company to his offshore portfolio?"

#user_context = "Can cash balances for individuals who are no longer tax resident be transferred offshore?"

excon.user_provides_input(user_context, threshold=0.15, model_to_use=model_to_use, temperature=0, max_tokens=300)



INFO:src.chat_bot:user: A non-resident wants to buy residential property in South Africa. What are the restrictions around doing this?
DEV:src.chat_bot:#################   Similarity Search       #################
DEV:src.chat_bot:--   Relevant Definitions
DEV:src.chat_bot:0.1282: (       all): Resident means any person (i.e. a natural person or legal entity) who has taken up permanent residence, is domiciled or registered in South Africa. For the purpose of the Authorised Dealer Manual, this excludes any approved offshore investments held by South African residents outside the CMA. However, such entities are still subject to exchange control rules and the Regulations. 
DEV:src.chat_bot:0.1438: (       all): Resident temporarily abroad means any resident who has departed from South Africa to any country outside the CMA with no intention of taking up residence or who has not been granted permanent residence in another country, excluding those residents who are abroad on holiday or busin

assistant: Non-residents are generally allowed to purchase residential property in South Africa. However, there are some restrictions and requirements that need to be met. According to section I.1(E)(i) of the Exchange Control Manual for Authorised Dealers, non-residents can be granted local financial assistance facilities for the acquisition of residential property, subject to normal lending criteria. 

Additionally, any facility being made available to a non-resident must be secured by an unencumbered Rand deposit or Rand-based asset of equal or higher value. The facility granted to the non-resident should not cause the borrower to exceed 100% of the Rand value of funds introduced from abroad and invested locally. 

It is important to note that facilities granted for the acquisition of fixed property cannot be increased based on a revaluation of the property at any stage. 

Reference: Exchange Control Manual for Authorised Dealers, Section I.1(E)(i)


In [30]:
flag, response = excon.resource_augmented_query(user_context, model_to_use = model_to_use, temperature = 0.0, max_tokens = 512, df_definitions = df_definitions, df_search_sections = df_search_sections)
print(f"Flag: {flag}")
print(f'Response: {response}')


{'role': 'system', 'content': "You are attempting to answer questions from an Authorised Dealer based only on the relevant documentation provided. You have only three options:\n1) Answer the question. If you do this, your must preface to response with the word 'ANSWER:'. If possible also provide a reference to the relevant documentation for the user to cross-check. Use this if you are sure about your answer.\n2) Request additional documentation. If, in the body of the relevant documentation, is a reference to another section of the document that is directly relevant, respond with the word 'SECTION:' followed by the section reference which looks like A.1(A)(i)(aa). \n3) State 'NONE:' (and nothing else) in all cases where you are not confident about either of the first two options\nPotentially relevant document section(s):\nIntroduction \n    In terms of Regulation 2(2), an Authorised Dealer shall not buy, borrow, receive, sell, lend or deliver any foreign currency or gold except for suc

In [12]:
print("### User Input")
print(user_context)
print("\n")

message = excon.messages[-1]["content"]
print('### Response')
print(message)
print("\n")

print("### Reference")
reference = message.split('\n')[-1]
print(excon.get_regulation_detail(reference))


### User Input
A non-resident wants to buy residential property in South Africa. What are the restrictions around doing this?


### Response
Non-residents are generally allowed to purchase residential property in South Africa. However, there are some restrictions and requirements that need to be met. According to section I.1(E)(i) of the Exchange Control Manual for Authorised Dealers, non-residents can be granted local financial assistance facilities for the acquisition of residential property, subject to normal lending criteria. 

Additionally, any facility being made available to a non-resident must be secured by an unencumbered Rand deposit or Rand-based asset of equal or higher value. The facility granted to the non-resident should not cause the borrower to exceed 100% of the Rand value of funds introduced from abroad and invested locally. 

It is important to note that facilities granted for the acquisition of fixed property cannot be increased based on a revaluation of the proper

In [9]:
excon.get_regulation_detail('I.(1)(E)(ii)')

'No section could be found with the reference I.(E)(ii)'

In [4]:
#user_context = "What is a CFC Account?"
#user_context = "I am submitting documentation to the financial surveillance department. What information do I need to include in my submission?" # (A.3 (B))
#user_context = "Can a bank provide foreign currency denominated facilities to a South African Corporate?" #  (A.3 (B))
#user_context = "For how long does the approval for a facility remain valid?" #  (A.3 (B))
#user_context = "Other than authorised transaction requests, are there requirements for authorised dealers to submit information to the financial surveillance department?" #  (A.3 (B))
#user_context = "I have been asked to act on an approval issued to another bank. Can I do that?" #  (A.3 (B))
#####user_context = "--- Same as above but use "Authorised Dealer" instead of "Bank"
#user_context = "If I am concerned that as client may in breach of regulations, what should I do?" #  (A.3 (B))
#user_context = "What constitutes 'documentary evidence' for a foreign currency transaction?" #  (A.3 (C))
#user_context = "Is Swaziland part of the CMA?" #  (A.3 (E))
#user_context = "A client wants to buy Namibian Dollars. What approvals do they need before they can do this?" #  (A.3 (E))
#user_context = "A person from Lesotho wants to buy dollars from me. Can I sell them the dollars?" #  (A.3 (E))
#user_context = "A person from Namibia wants to buy rand from me. Can I sell them the rand?" #  (A.3 (E))
#user_context = "A treasury management company wants to trade dollars with me. What do I need to check before I can do this?" #  (A.4 (B))
#user_context = "A client wants to import goods and pay in foreign currency. What documentation do I need before I can assist with the transaction" #  (B.1 (B))
#user_context = "Over and above the price of the goods, what additional costs can be covered in foreign currency?" #  (B.1 (C))
#user_context = "Can advance payment be made in a foreign currency to cover imported goods?" #  (B.1 (D)(i))

#user_context = "Can advance payment be made in a foreign currency to cover imported capital goods?" #  (B.1 (D)(ii))
#user_context = "When paying for imports in cash, what documentation is required before providing the foreign currency?" #  (B.1 (D)(iii))

#user_context = "Can imported goods be financed over more than 12 months?" #  (B.1 (D)(iv))
#user_context = "How much money can an individual invest internationally?" #  ?
#user_context = "How much money can an individual take offshore?" #  ?

#user_context = "A media company want to list offshore. Are there specific conditions for this?"
# user_context = "How many rands can a resident take offshore in cash?"
#user_context = "Can I grant a foreign company a Rand facility to pay for goods they wish to purchase from a South African exporter?"
#user_context = "Can I support a non-residential entity with short-term rand trade finance for their exports from South Africa?"
#user_context = "Hi"
#user_context = "Who can trade gold?"
#user_context = "are there specific exchange control rules for religious organizations?"
#user_context = "How much foreign currency can a charity take offshore each year?"

